In [1]:
# Spark Imports
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [2]:
# Dates and Timestamps specific functions (or could use with f.*)
from pyspark.sql.functions import (dayofmonth, hour, dayofyear,
                                   month, year, weekofyear,
                                   format_number, date_format)

In [3]:
# Start the Spark session.
spark = SparkSession.builder.appName('dates').getOrCreate()

In [4]:
# We can infer the schema/types (only in CSV), and header tells us
# that the first row are the names of the columns.
df = spark.read.csv('Data/appl_stock.csv', 
                    header=True, inferSchema=True)

In [5]:
# Remember .head() will always return an array.
row = df.head(1)[0]
row

Row(Date=datetime.datetime(2010, 1, 4, 0, 0), Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039)

In [6]:
df.select(['Date', 'Open']).show()

+-------------------+------------------+
|               Date|              Open|
+-------------------+------------------+
|2010-01-04 00:00:00|        213.429998|
|2010-01-05 00:00:00|        214.599998|
|2010-01-06 00:00:00|        214.379993|
|2010-01-07 00:00:00|            211.75|
|2010-01-08 00:00:00|        210.299994|
|2010-01-11 00:00:00|212.79999700000002|
|2010-01-12 00:00:00|209.18999499999998|
|2010-01-13 00:00:00|        207.870005|
|2010-01-14 00:00:00|210.11000299999998|
|2010-01-15 00:00:00|210.92999500000002|
|2010-01-19 00:00:00|        208.330002|
|2010-01-20 00:00:00|        214.910006|
|2010-01-21 00:00:00|        212.079994|
|2010-01-22 00:00:00|206.78000600000001|
|2010-01-25 00:00:00|202.51000200000001|
|2010-01-26 00:00:00|205.95000100000001|
|2010-01-27 00:00:00|        206.849995|
|2010-01-28 00:00:00|        204.930004|
|2010-01-29 00:00:00|        201.079996|
|2010-02-01 00:00:00|192.36999699999998|
+-------------------+------------------+
only showing top

In [7]:
# `dayofmonth`: the number of the day in the month.
# `hour`: the hour of the day.
# `dayofyear`: the number of the day in the year.
# `month`: the number of the month (1-index).
# `year`: the number of the year.
# `weekofyear`: the number of the week in the year.
df.select(dayofyear(f.col('Date'))).where(dayofmonth(f.col('Date')) == 1).show()

+---------------+
|dayofyear(Date)|
+---------------+
|             32|
|             60|
|             91|
|            152|
|            182|
|            244|
|            274|
|            305|
|            335|
|             32|
|             60|
|             91|
|            152|
|            182|
|            213|
|            244|
|            305|
|            335|
|             32|
|             61|
+---------------+
only showing top 20 rows



In [8]:
# Let's find the average closing price per year.
# df.select(year(f.col('Date'))).show()
new_df = df.withColumn("Year", year(f.col('Date')))

In [9]:
# Let's get only these two columns to prevent calculating mean
# of a lot of other columns unnecessarily.
new_df = new_df.select("Year", "Close")

In [10]:
# Calculate the mean only of the Close columnt otherwise just calling
# .mean() will do it in all numeric columns including year.
result = new_df.groupBy("Year").agg({"Close" : "avg"})

In [11]:
# Rename the column with the average of closing price more readable.
# result = result.withColumnRenamed("avg(Close)", "AVG Closing Price")
# Fix to two significant digits, the alias can be done there instead
# of calling the column renamed function, whatever you like.
result_rounded = result.select(['Year', 
               format_number('avg(Close)', 2)
               .alias("AVG Closing Price")])

In [12]:
# Check the average closing price of the stock per year!
result_rounded.show()

+----+-----------------+
|Year|AVG Closing Price|
+----+-----------------+
|2015|           120.04|
|2013|           472.63|
|2014|           295.40|
|2012|           576.05|
|2016|           104.60|
|2010|           259.84|
|2011|           364.00|
+----+-----------------+



In [13]:
# Also can take a datetime element and convert it, let's say,
# extract only month and day from a whole datetime object.
df.select(date_format(f.col("Date"), 'MM-dd')).show()

+------------------------+
|date_format(Date, MM-dd)|
+------------------------+
|                   01-04|
|                   01-05|
|                   01-06|
|                   01-07|
|                   01-08|
|                   01-11|
|                   01-12|
|                   01-13|
|                   01-14|
|                   01-15|
|                   01-19|
|                   01-20|
|                   01-21|
|                   01-22|
|                   01-25|
|                   01-26|
|                   01-27|
|                   01-28|
|                   01-29|
|                   02-01|
+------------------------+
only showing top 20 rows

